<a href="https://colab.research.google.com/github/dgullate/Curso-IE/blob/master/IMDB_word_embeddings_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import pandas as pd
import nltk
nltk.download('stopwords')
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Loading the data

First, we read in the data with pandas, as we did in Part 1. Unlike Part 1, we now use unlabeledTrain.tsv, which contains 50,000 additional reviews with no labels. When we built the Bag of Words model in Part 1, extra unlabeled training reviews were not useful. However, since Word2Vec can learn from unlabeled data, these extra 50,000 reviews can now be used.

In [0]:
train_data_path='https://github.com/dgullate/Curso-IE/blob/master/Week_4/IMDB/data/labeledTrainData.tsv?raw=true'
test_data_path='https://github.com/dgullate/Curso-IE/blob/master/Week_4/IMDB/data/testData.tsv?raw=true'
unlabeled_data_path='https://github.com/dgullate/Curso-IE/blob/master/Week_4/IMDB/data/unlabeledTrainData.tsv?raw=true'
train = pd.read_csv(train_data_path, header=0, delimiter="\t", quoting=3)
unlabeled_train = pd.read_csv(unlabeled_data_path, header=0, delimiter="\t", quoting=3)
test = pd.read_csv(test_data_path, header=0, delimiter="\t", quoting=3)

## Data cleaning and preprocessing

The functions we write to clean the data are also similar to Part 1, although now there are a couple of differences. First, to train Word2Vec it is better not to remove stop words because the algorithm relies on the broader context of the sentence in order to produce high-quality word vectors. For this reason, we will make stop word removal optional in the functions below. It also might be better not to remove numbers, but we leave that as an exercise for the reader.

In [0]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

Next, we want a specific input format. Word2Vec expects single sentences, each one as a list of words. In other words, the input format is a list of lists.

It is not at all straightforward how to split a paragraph into sentences. There are all kinds of gotchas in natural language. English sentences can end with "?", "!", """, or ".", among other things, and spacing and capitalization are not reliable guides either. For this reason, we'll use NLTK's punkt tokenizer for sentence splitting. In order to use this, you will need to install NLTK and use nltk.download() to download the relevant training file for punkt.

In [4]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download('punkt')   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence,remove_stopwords))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Let us look at one review and the output of this last function on it:

In [5]:
sentences= review_to_sentences(train['review'][7],tokenizer)
for sentence in sentences:
  print(sentence)

['i', 'watched', 'this', 'video', 'at', 'a', 'friend', 's', 'house']
['i', 'm', 'glad', 'i', 'did', 'not', 'waste', 'money', 'buying', 'this', 'one']
['the', 'video', 'cover', 'has', 'a', 'scene', 'from', 'the', 'movie', 'capricorn', 'one']
['the', 'movie', 'starts', 'out', 'with', 'several', 'clips', 'of', 'rocket', 'blow', 'ups', 'most', 'not', 'related', 'to', 'manned', 'flight']
['sibrel', 's', 'smoking', 'gun', 'is', 'a', 'short', 'video', 'clip', 'of', 'the', 'astronauts', 'preparing', 'a', 'video', 'broadcast']
['he', 'edits', 'in', 'his', 'own', 'voice', 'over', 'instead', 'of', 'letting', 'us', 'listen', 'to', 'what', 'the', 'crew', 'had', 'to', 'say']
['the', 'video', 'curiously', 'ends', 'with', 'a', 'showing', 'of', 'the', 'zapruder', 'film']
['his', 'claims', 'about', 'radiation', 'shielding', 'star', 'photography', 'and', 'others', 'lead', 'me', 'to', 'believe', 'is', 'he', 'extremely', 'ignorant', 'or', 'has', 'some', 'sort', 'of', 'ax', 'to', 'grind', 'against', 'nasa',

Now we can apply this function to prepare our data for input to Word2Vec (this will take a couple minutes):

In [6]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
n=0
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)
    n+=1
    if n%5000==0:
      print('Already parsed {} files'.format(n))
n=0
print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)
    n+=1
    if n%5000==0:
      print('Already parsed {} files'.format(n))


Parsing sentences from training set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Already parsed 5000 files
Already parsed 10000 files
Already parsed 15000 files
Already parsed 20000 files
Already parsed 25000 files
Parsing sentences from unlabeled set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Already parsed 5000 files
Already parsed 10000 files
Already parsed 15000 files
Already parsed 20000 files


...

Already parsed 35000 files


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.youtube.com/watch?v=a0KSqelmgN8"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Already parsed 40000 files
Already parsed 45000 files


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://jake-weird.blogspot.com/2007/08/beneath.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Already parsed 50000 files


You may get a few warnings from BeautifulSoup about URLs in the sentences. These are nothing to worry about (although you may want to consider removing URLs when cleaning the text). 

We can take a look at the output to see how this differs from Part 1:

In [11]:
print(len(sentences))
sentences[0]

795538


['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 've',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again']

## Training word2vec on our dataset

With the list of nicely parsed sentences, we're ready to train the model. There are a number of parameter choices that affect the run time and the quality of the final model that is produced. For details on the algorithms below, see the word2vec API documentation as well as the Google documentation. 

1. **Architecture**: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.

2. **Training algorithm**: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.

3. **Word vector dimensionality**: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.

4. **Context / window size**: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).

5. **Worker threads**: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.

6. **Minimum word count**: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

Choosing parameters is not easy, but once we have chosen our parameters, creating a Word2Vec model is straightforward:

In [12]:
## NOTE: This calculation takes aprox. 4 min. on Google colab

# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-05-20 16:58:44,269 : INFO : 'pattern' package not found; tag filters are not available for English
2019-05-20 16:58:44,281 : INFO : collecting all words and their counts
2019-05-20 16:58:44,282 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-05-20 16:58:44,345 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-05-20 16:58:44,409 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2019-05-20 16:58:44,468 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types


Training model...


2019-05-20 16:58:44,531 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2019-05-20 16:58:44,588 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-05-20 16:58:44,650 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-05-20 16:58:44,709 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-05-20 16:58:44,769 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-05-20 16:58:44,828 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-05-20 16:58:44,888 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-05-20 16:58:44,948 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2019-05-20 16:58:45,008 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keepin

## Exploring the Model Results
Let's take a look at the model we created out of our 75,000 training reviews.

The `doesnt_match` function will try to deduce which word in a set is most dissimilar from the others:

In [13]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

Our model is capable of distinguishing differences in meaning! It knows that men, women and children are more similar to each other than they are to kitchens. More exploration shows that the model is sensitive to more subtle differences in meaning, such as differences between countries and cities:

In [14]:
model.doesnt_match("france england spain madrid".split())

'madrid'

We can also use the "most_similar" function to get insight into the model's word clusters:



In [37]:
model.most_similar("man")

[('woman', 0.6234472990036011),
 ('lady', 0.6112213134765625),
 ('lad', 0.5950173139572144),
 ('monk', 0.55019611120224),
 ('chap', 0.536170482635498),
 ('guy', 0.5299067497253418),
 ('men', 0.5216111540794373),
 ('soldier', 0.5134493112564087),
 ('boy', 0.5083514451980591),
 ('millionaire', 0.5068236589431763)]

In [15]:
model.most_similar("philosophy")

[('psychology', 0.6522006988525391),
 ('ethics', 0.6438441276550293),
 ('religion', 0.6426934599876404),
 ('alienation', 0.6314270496368408),
 ('spiritual', 0.6280637979507446),
 ('sexuality', 0.623652458190918),
 ('ideals', 0.6229167580604553),
 ('homosexuality', 0.6200724840164185),
 ('cultural', 0.6139366626739502),
 ('philosophical', 0.613634467124939)]

In [16]:
model.most_similar("queen")

[('princess', 0.6843096613883972),
 ('bride', 0.6176869869232178),
 ('belle', 0.6127505302429199),
 ('victoria', 0.5895715951919556),
 ('maria', 0.5855535864830017),
 ('maid', 0.5800403356552124),
 ('prince', 0.5783246755599976),
 ('mistress', 0.5782764554023743),
 ('goddess', 0.565935492515564),
 ('sybil', 0.5643234252929688)]

Or, more relevant for sentiment analysis:

In [17]:
model.most_similar("awful")

[('terrible', 0.7762552499771118),
 ('atrocious', 0.7497725486755371),
 ('horrible', 0.7409049272537231),
 ('dreadful', 0.7130718231201172),
 ('abysmal', 0.707388699054718),
 ('horrendous', 0.70050048828125),
 ('horrid', 0.6918109059333801),
 ('appalling', 0.671273410320282),
 ('lousy', 0.6340524554252625),
 ('amateurish', 0.624087929725647)]

In [23]:
model.doesnt_match("france england spain mexico".split())

'mexico'

## Numeric Representations of Words
Now that we have a trained model with some semantic understanding of words, how should we use it? If you look beneath the hood, the Word2Vec model trained above consists of a feature vector for each word in the vocabulary, stored in a numpy array called "syn0":

In [24]:
# Load the model that we created in Part 2
from gensim.models import Word2Vec
model = Word2Vec.load("300features_40minwords_10context")

2019-05-20 17:05:21,109 : INFO : loading Word2Vec object from 300features_40minwords_10context
2019-05-20 17:05:21,111 : WARNING : this function is deprecated, use smart_open.open instead
2019-05-20 17:05:21,471 : INFO : loading wv recursively from 300features_40minwords_10context.wv.* with mmap=None
2019-05-20 17:05:21,472 : INFO : setting ignored attribute vectors_norm to None
2019-05-20 17:05:21,473 : INFO : loading vocabulary recursively from 300features_40minwords_10context.vocabulary.* with mmap=None
2019-05-20 17:05:21,474 : INFO : loading trainables recursively from 300features_40minwords_10context.trainables.* with mmap=None
2019-05-20 17:05:21,475 : INFO : setting ignored attribute cum_table to None
2019-05-20 17:05:21,476 : INFO : loaded 300features_40minwords_10context


The model is a big matrix with as many rows as words in our dictionary (), and as many columns as the latent dimension (the dimension of the embeddings, which we chose to be 300).


In [25]:
print(model.wv.syn0.shape)
len(model.wv.syn0[0])

(16490, 300)


300

Now we can retrieve the embedding of any word in our dictionary, as a dense vector  in this 300-dimensional space.

In [26]:
model['apple']

array([-2.45181546e-02, -2.71881502e-02,  7.83065520e-03,  4.11549844e-02,
        1.01373030e-03, -1.54032242e-02, -8.20618719e-02, -5.07392474e-02,
       -7.38434047e-02, -5.61182247e-03,  1.49452705e-02,  8.50087106e-02,
       -8.10130220e-03, -6.22096413e-04, -4.75194678e-02,  3.89971547e-02,
        3.46173230e-03,  4.73913271e-03,  1.08233087e-01,  1.72881354e-02,
       -5.63459881e-02, -6.82060868e-02,  1.18901990e-02,  3.27156298e-02,
        5.89986555e-02,  1.15192860e-01,  8.59185413e-04, -1.02000706e-01,
       -3.39285582e-02,  3.05047575e-02, -4.64416407e-02,  2.29347367e-02,
       -2.56928545e-03, -3.20303366e-02, -3.21944095e-02, -1.37880230e-02,
        4.47443575e-02, -3.71000506e-02,  1.94447245e-02,  6.35240274e-03,
       -4.74460758e-02,  9.31219608e-02,  6.74689412e-02,  1.21563263e-01,
       -7.09020719e-02, -1.31394297e-01,  2.88513564e-02, -5.46351410e-02,
       -4.63539995e-02,  4.19577770e-02, -1.25627117e-02, -7.03739077e-02,
       -8.53957310e-02, -

### From Words To Paragraphs, Attempt 1: Vector Averaging
One challenge with the IMDB dataset is the variable-length reviews. We need to find a way to take individual word vectors and transform them into a feature set that is the same length for every review.

Since each word is a vector in 300-dimensional space, we can use vector operations to combine the words in each review. One method we tried was to simply average the word vectors in a given review (for this purpose, we removed stop words, which would just add noise).

The following code averages the feature vectors, building on our code from Part 2. 

In [0]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print ("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
       #
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

As an example, let us calculate the embedding for a single review:

In [43]:
review_example=review_to_wordlist( train['review'][0],remove_stopwords=True )
print('This review has {} words'.format(len(review_example)))
emb_review=makeFeatureVec(review_example, model, num_features)
emb_review.shape

This review has 219 words


(300,)

Now, we can call these functions to create average vectors for each review. The following operations will take a few minutes:

In [46]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review,remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

## Applying classification models

In Part 1, each review was encoded as a sparse vector of length 5.000 (size of vocabulary) and integer entries specifying the number of occurrences each word in the review. In Part 2, we encode each review as a dense vector of dimension 300, which is an average of the word embeddings of each word in the review.

Now we can use this encoding to train our favourite classifiers and look at their performance.

### Logistic regression

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(trainDataVecs, train["sentiment"], test_size=0.25, random_state=42)

In [48]:
X_train.shape

(18750, 300)

In [49]:
from sklearn.linear_model import LogisticRegression

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

clf=LogisticRegression()
clf.fit(X_train,y_train)
print('Accuracy on the training set: ',clf.score(X_train,y_train))
print('Accuracy on the validation set: ',clf.score(X_val,y_val))

Accuracy on the training set:  0.8552
Accuracy on the validation set:  0.8544


Observe that, as opposed to Part 1 where we had many more features, the default Logistic Regression classifier does not overfit.

In [50]:
from sklearn.model_selection import GridSearchCV
parameters = {'C':[0.1,1,10],'penalty':['l1','l2']}
clf=LogisticRegression( )
grid_clf = GridSearchCV(clf, parameters, cv=3)
grid_clf.fit(X_train,y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [51]:
print('Accuracy on the training set: ',grid_clf.score(X_train,y_train))
print('Accuracy on the validation set: ',grid_clf.score(X_val,y_val))

Accuracy on the training set:  0.8776
Accuracy on the validation set:  0.87456


A grid search improves 2 points on the accuracy result. Let's look at the best parameters

In [52]:
grid_clf.best_estimator_

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

### Random Forest

In [55]:
print ("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( X_train, y_train )

Training the random forest...


In [56]:
forest.score(X_val,y_val)

0.83872

## Comparison between Bag of Words and Word embeddings

Here we compare performance of classifiers (Logistic Regression and Random Forest) with two different review encodings: BoW and word embeddings.
As a conclusion, performance is similar, no huge differences, but we did improve 2% accuracy with regularized LR on the word embeddings.

**Question:** Can you explain why there is not such a huge difference in performance ? Are the two review encodings really that different ?

**Improvements:** A standard way of weighting word vectors is to apply "tf-idf" weights, which measure how important a given word is within a given set of documents. One way to extract tf-idf weights in Python is by using scikit-learn's TfidfVectorizer, which has an interface similar to the CountVectorizer that we used in Part 1. 